# Importanto bibliotecas e Definindo diretórios

In [1]:
import os
import cv2
import numpy as np
import joblib
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, classification_report
from tqdm import tqdm

data_dir = f"./data"
#corel-1k
dataset_dir = f"{data_dir}/dataset"
images_dir = f"{dataset_dir}/images"
#caltech256
dataset_1_dir = f"{data_dir}/dataset_1"
images_1_dir = f"{dataset_1_dir}/images"
#cifar100
dataset_2_dir = f"{data_dir}/dataset_2"
images_2_dir = f"{dataset_2_dir}/images"
#cifar10
dataset_3_dir = f"{data_dir}/dataset_3"
images_3_dir = f"{dataset_3_dir}/images"

# Funções p/ escala de cinza

## Geração de histogramas

In [2]:
def obter_vizinhos(matriz_de_intensidade, linha, coluna):
    # O objetivo dessa função é retornar uma lista contendo a intensidade dos 9 vizinhos do pixel observado

    #Esse método de partição da matriz foi escolhido para manter a analise dos pixels vizinhos
    #dentro dos limites da matriz de intensidade, evitando assim valores negativos ou fora do shape.
    vizinhos = matriz_de_intensidade[max(0, linha-1):min(matriz_de_intensidade.shape[0], linha+2),
                             max(0, coluna-1):min(matriz_de_intensidade.shape[1], coluna+2)]

    #Transforma a matriz particionada em lista e remove o pixel do centro
    lista_de_vizinhos = vizinhos.flatten().tolist()
    lista_de_vizinhos.remove(matriz_de_intensidade[linha, coluna])

    return lista_de_vizinhos

In [3]:
def aplica_regras(lista_de_vizinhos, intensidade_pixel_central, estado_pixel_central):
    #O objetivo dessa função é aplicar as regras do jogo da vida de Conway no pixel observado e retornar seu estado.
    # 1 = vivo, 0 = morto

    #Conta o número de vizinhos vivos com a mesma intensidade que o pixel central
    vizinhos_iguais = lista_de_vizinhos.count(intensidade_pixel_central)

    #Regra 1: A célula viva com dois ou três vizinhos vivos sobrevive
    if estado_pixel_central == 1 and (vizinhos_iguais == 2 or vizinhos_iguais == 3):
        return 1

    #Regra 2: A célula viva com menos de dois vizinhos vivos morre (subpopulação)
    elif estado_pixel_central == 1 and vizinhos_iguais < 2:
        return 0

    #Regra 3: A célula viva com mais de três vizinhos vivos morre (superpopulação)
    elif estado_pixel_central == 1 and vizinhos_iguais > 3:
        return 0

    #Regra 4: A célula morta com exatamente três vizinhos vivos se torna viva (resurreição)
    elif estado_pixel_central == 0 and vizinhos_iguais == 3:
        return 1

    #Pra todos os outros casos, a célula permanece no mesmo estado
    else:
        return estado_pixel_central

In [4]:
def percorre_imagem_aplicando_regras(matriz_de_estados, matriz_de_intensidade):
    # O objetivo dessa função é percorrer a imagem, chamar a função para obter os vizinhos e aplicar as regras

    linhas, colunas = matriz_de_intensidade.shape
    for linha in range(linhas):
        for coluna in range(colunas):
            #Obtem os vizinhos do pixel atual
            lista_de_vizinhos = obter_vizinhos(matriz_de_intensidade, linha, coluna)
            #Aplica as regras do jogo da vida no pixel atual (atualiza a matriz de estado inicial)
            matriz_de_estados[linha, coluna] = aplica_regras(lista_de_vizinhos, matriz_de_intensidade[linha, coluna], matriz_de_estados[linha, coluna])

    return matriz_de_estados

In [5]:
def gera_histogramas(imagem_cinza):
    #O objetivo dessa função é criar as matrizes de intensidade e as de estado inicial para cada imagem
    #Após aplicar as regras cria os histogramas

    #Transforma a imagem em uma matriz de intensidade
    matriz_de_intensidade = np.array(imagem_cinza)

    #Cria as matrizes de estados iniciais
    matriz_de_estados_phi = np.ones(matriz_de_intensidade.shape, dtype=int) #todos vivos
    matriz_de_estados_psi = np.zeros(matriz_de_intensidade.shape, dtype=int) #todos mortos

    #Aplica as regras do jogo da vida e atualiza as matrizes de estado inicial
    matriz_de_estados_phi = percorre_imagem_aplicando_regras(matriz_de_estados_phi, matriz_de_intensidade)
    matriz_de_estados_psi = percorre_imagem_aplicando_regras(matriz_de_estados_psi, matriz_de_intensidade)

    #As matrizes são convertidas em listas
    #Phi -> estado inicial = vivo
    phi_vivos = matriz_de_intensidade.flatten()[matriz_de_estados_phi.flatten() == 1] #se manteram vivos
    phi_mortos = matriz_de_intensidade.flatten()[matriz_de_estados_phi.flatten() == 0] #morreram

    #Psi -> estado inicial = morto
    psi_vivos = matriz_de_intensidade.flatten()[matriz_de_estados_psi.flatten() == 1] #ressuscitaram
    psi_mortos = matriz_de_intensidade.flatten()[matriz_de_estados_psi.flatten() == 0] #se manteram mortos

    #Cria os histogramas
    hist_phi_vivos, _ = np.histogram(phi_vivos, bins=256, range=(0, 256))
    hist_phi_mortos, _ = np.histogram(phi_mortos, bins=256, range=(0, 256))
    hist_psi_vivos, _ = np.histogram(psi_vivos, bins=256, range=(0, 256))
    hist_psi_mortos, _ = np.histogram(psi_mortos, bins=256, range=(0, 256))

    return hist_phi_vivos, hist_phi_mortos, hist_psi_vivos, hist_psi_mortos

In [6]:
def gerador_histogramas(dataset_dir, escala):
    #Funcao principal: faz a chamada das funções acima e salva os histogramas

    print("Gerando histogramas")
    images_dir = f"{dataset_dir}/images"
    histograms_dir = f"{dataset_dir}/histograms_grayscale"

    # Cria as pastas para salvar os histogramas mantendo o padrão das classes do dataset
    os.makedirs(histograms_dir, exist_ok=True)
    classes = [conteudo_item for conteudo_item in os.listdir(images_dir) if os.path.isdir(os.path.join(images_dir, conteudo_item))]
    for classe in classes:
        dir_pastas = os.path.join(histograms_dir, classe)
        os.makedirs(dir_pastas, exist_ok=True)


    #Esse loop pega a imagem, gera seus histogramas e salva com base no nome da classe e da imagem
    for classe in os.listdir(images_dir):
        dir_classe = f"{images_dir}/{classe}"
        classe = f"{classe}"
        
        lista_arquivos = os.listdir(dir_classe)
        num_arquivos = len(lista_arquivos)
        
        progress_bar = tqdm(total=num_arquivos, desc= classe)
        
        for imagem in os.listdir(dir_classe):
            imagem_path = f"{images_dir}/{classe}/{imagem}"

            imagem_cinza = cv2.imread(imagem_path, cv2.IMREAD_GRAYSCALE) #Lendo em escala de cinza
            imagem_cinza = cv2.resize(imagem_cinza, (escala,escala)) #Padronizando tamanho das imagens
            
            hist_phi_vivos, hist_phi_mortos, hist_psi_vivos, hist_psi_mortos = gera_histogramas(imagem_cinza)

            img_name = imagem.split(".")[0]
            file_path = os.path.join(f"{histograms_dir}/{classe}", f"{img_name}_phi_vivos.pkl")
            joblib.dump(hist_phi_vivos, file_path)
            file_path = os.path.join(f"{histograms_dir}/{classe}", f"{img_name}_phi_mortos.pkl")
            joblib.dump(hist_phi_mortos, file_path)
            file_path = os.path.join(f"{histograms_dir}/{classe}", f"{img_name}_psi_vivos.pkl")
            joblib.dump(hist_psi_vivos, file_path)
            file_path = os.path.join(f"{histograms_dir}/{classe}", f"{img_name}_psi_mortos.pkl")
            joblib.dump(hist_psi_mortos, file_path)
            progress_bar.update(1)

        progress_bar.close()

    print("Progresso concluído")

## Preparação de dados

In [7]:
def carrega_hist(imagem_path):
    #O objetivo dessa função é carregar os histogramas de determinada imagem

    #./data/dataset/images/classe/imagem.jpg
    imagem_path = imagem_path.split("/")
    dataset, classe, imagem = imagem_path[2], imagem_path[4], imagem_path[5]
    imagem = imagem.split(".")[0]

    histogramas_dir = f"./data/{dataset}/histograms_grayscale"

    try:
        #Buscando os histogramas no diretorio
        hist_phi_vivos = joblib.load(f"{histogramas_dir}/{classe}/{imagem}_phi_vivos.pkl")
        hist_phi_mortos = joblib.load(f"{histogramas_dir}/{classe}/{imagem}_phi_mortos.pkl")
        hist_psi_vivos = joblib.load(f"{histogramas_dir}/{classe}/{imagem}_psi_vivos.pkl")
        hist_psi_mortos = joblib.load(f"{histogramas_dir}/{classe}/{imagem}_psi_mortos.pkl")

        return hist_phi_vivos, hist_phi_mortos, hist_psi_vivos, hist_psi_mortos

    except:
        print(imagem_path)
        print("Os histogramas não foram encontrados. Verifique os diretórios e se os histogramas foram criados corretamente.")

In [8]:
def retorna_combinacao(imagem_path, combinacao):
    # O objetivo dessa funcao é criar combinacoes com os histogramas da imagem

    hist_phi_vivos, hist_phi_mortos, hist_psi_vivos, hist_psi_mortos = carrega_hist(imagem_path)
    if combinacao == 0:
        return hist_phi_vivos
    elif combinacao == 1:
        return hist_phi_mortos
    elif combinacao == 2:
        return hist_psi_vivos
    elif combinacao == 3:
        return hist_psi_mortos
    elif combinacao == 4:
        return np.concatenate([hist_phi_vivos, hist_phi_mortos]) #histogramas phi
    elif combinacao == 5:
        return np.concatenate([hist_psi_vivos, hist_psi_mortos]) #histogramas psi
    elif combinacao == 6:
        return np.concatenate([hist_phi_vivos, hist_psi_vivos]) #histogramas vivos
    elif combinacao == 7:
        return np.concatenate([hist_phi_mortos, hist_psi_mortos]) #histogramas mortos
    elif combinacao == 8:
        return np.concatenate([np.concatenate([hist_phi_vivos, hist_phi_mortos]),
                               np.concatenate([hist_psi_vivos, hist_psi_mortos])])
        #histogramas phi e psi combinados

In [9]:
# Prepara os dados para treinar o modelo
def retorna_dados(images_dir):
    histogramas = []
    rotulos = []

    print("Carregando histogramas")
    for classe in os.listdir(images_dir):
        dir_classe = f"{images_dir}/{classe}"
        classe = f"{classe}"
        
        lista_arquivos = os.listdir(dir_classe)
        num_arquivos = len(lista_arquivos)
        progress_bar = tqdm(total=num_arquivos, desc= classe)
        
        for imagem in os.listdir(dir_classe):
            imagem_path = f"{images_dir}/{classe}/{imagem}"
    
            #Pode alterar a combinacao
            combinacao = retorna_combinacao(imagem_path, 8)
    
            #Lista dos histogramas
            histogramas.append(combinacao)
            #lista da classe assossiada ao histograma
            rotulos.append(classe)
            progress_bar.update(1)
        
        progress_bar.close()
        
    print("Progresso concluído")
    return histogramas, rotulos

# Funções p/ RGB

## Geração de histogramas

In [10]:
def obter_vizinhos_rgb(matriz_de_intensidade, linha, coluna):
    # O objetivo dessa função é retornar uma lista contendo a intensidade dos 9 vizinhos do pixel observado

    #Esse método de partição da matriz foi escolhido para manter a analise dos pixels vizinhos
    #dentro dos limites da matriz de intensidade, evitando assim valores negativos ou fora do shape.
    vizinhos = matriz_de_intensidade[max(0, linha-1):min(matriz_de_intensidade.shape[0], linha+2),
                             max(0, coluna-1):min(matriz_de_intensidade.shape[1], coluna+2)]

    #Transforma a matriz particionada em lista e remove o pixel do centro
    lista_de_vizinhos = vizinhos.flatten().tolist()
    lista_de_vizinhos.remove(matriz_de_intensidade[linha, coluna])

    return lista_de_vizinhos

In [11]:
def aplica_regras_rgb(lista_de_vizinhos, intensidade_pixel_central, estado_pixel_central):
    #O objetivo dessa função é aplicar as regras do jogo da vida de Conway no pixel observado e retornar seu estado.
    # 1 = vivo, 0 = morto

    #Conta o número de vizinhos vivos com a mesma intensidade que o pixel central
    vizinhos_iguais = lista_de_vizinhos.count(intensidade_pixel_central)

    #Regra 1: A célula viva com dois ou três vizinhos vivos sobrevive
    if estado_pixel_central == 1 and (vizinhos_iguais == 2 or vizinhos_iguais == 3):
        return 1

    #Regra 2: A célula viva com menos de dois vizinhos vivos morre (subpopulação)
    elif estado_pixel_central == 1 and vizinhos_iguais < 2:
        return 0

    #Regra 3: A célula viva com mais de três vizinhos vivos morre (superpopulação)
    elif estado_pixel_central == 1 and vizinhos_iguais > 3:
        return 0

    #Regra 4: A célula morta com exatamente três vizinhos vivos se torna viva (resurreição)
    elif estado_pixel_central == 0 and vizinhos_iguais == 3:
        return 1

    #Pra todos os outros casos, a célula permanece no mesmo estado
    else:
        return estado_pixel_central

In [12]:
def percorre_imagem_aplicando_regras_rgb(matriz_de_estados, matriz_de_intensidade):
    # O objetivo dessa função é percorrer a imagem, chamar a função para obter os vizinhos e aplicar as regras

    linhas, colunas = matriz_de_intensidade.shape
    for linha in range(linhas):
        for coluna in range(colunas):
            #Obtem os vizinhos do pixel atual
            lista_de_vizinhos = obter_vizinhos_rgb(matriz_de_intensidade, linha, coluna)
            #Aplica as regras do jogo da vida no pixel atual (atualiza a matriz de estado inicial)
            matriz_de_estados[linha, coluna] = aplica_regras_rgb(lista_de_vizinhos, matriz_de_intensidade[linha, coluna], matriz_de_estados[linha, coluna])

    return matriz_de_estados

In [13]:
def gera_histogramas_rgb(imagem):
    #O objetivo dessa função é criar as matrizes de intensidade e as de estado inicial para cada imagem
    #Após aplicar as regras cria os histogramas

    #Transforma a imagem em uma matriz de intensidade
    matriz_de_intensidade = np.array(imagem)

    #Cria as matrizes de estados iniciais
    matriz_de_estados_phi = np.ones(matriz_de_intensidade.shape, dtype=int) #todos vivos
    matriz_de_estados_psi = np.zeros(matriz_de_intensidade.shape, dtype=int) #todos mortos

    #Aplica as regras do jogo da vida e atualiza as matrizes de estado inicial
    matriz_de_estados_phi = percorre_imagem_aplicando_regras_rgb(matriz_de_estados_phi, matriz_de_intensidade)
    matriz_de_estados_psi = percorre_imagem_aplicando_regras_rgb(matriz_de_estados_psi, matriz_de_intensidade)

    #As matrizes são convertidas em listas
    #Phi -> estado inicial = vivo
    phi_vivos = matriz_de_intensidade.flatten()[matriz_de_estados_phi.flatten() == 1] #se manteram vivos
    phi_mortos = matriz_de_intensidade.flatten()[matriz_de_estados_phi.flatten() == 0] #morreram

    #Psi -> estado inicial = morto
    psi_vivos = matriz_de_intensidade.flatten()[matriz_de_estados_psi.flatten() == 1] #ressuscitaram
    psi_mortos = matriz_de_intensidade.flatten()[matriz_de_estados_psi.flatten() == 0] #se manteram mortos

    #Cria os histogramas
    hist_phi_vivos, _ = np.histogram(phi_vivos, bins=256, range=(0, 256))
    hist_phi_mortos, _ = np.histogram(phi_mortos, bins=256, range=(0, 256))
    hist_psi_vivos, _ = np.histogram(psi_vivos, bins=256, range=(0, 256))
    hist_psi_mortos, _ = np.histogram(psi_mortos, bins=256, range=(0, 256))

    return hist_phi_vivos, hist_phi_mortos, hist_psi_vivos, hist_psi_mortos

In [14]:
def gerador_histogramas_RGB(dataset_dir, escala):
    images_dir = f"{dataset_dir}/images"
    histograms_dir = f"{dataset_dir}/histograms_rgb"

    print("Gerando histogramas")
    #Criando diretorios das classes
    os.makedirs(histograms_dir, exist_ok=True)
    classes = [d for d in os.listdir(images_dir) if os.path.isdir(os.path.join(images_dir, d))]
    for classe in classes:
        os.makedirs(os.path.join(histograms_dir, classe), exist_ok=True)

    for classe in classes:
        dir_classe = os.path.join(images_dir, classe)

        lista_arquivos = os.listdir(dir_classe)
        num_arquivos = len(lista_arquivos)
        progress_bar = tqdm(total=num_arquivos, desc= classe)
        
        for imagem in os.listdir(dir_classe):
            imagem_path = os.path.join(dir_classe, imagem)
            imagem_rgb = cv2.imread(imagem_path, cv2.IMREAD_COLOR) #Lendo a imagem em BGR
            imagem_rgb = cv2.resize(imagem_rgb, (escala, escala))

            #Gera os histogramas para cada canal da imagem
            hist_phi_vivos_b, hist_phi_mortos_b, hist_psi_vivos_b, hist_psi_mortos_b = gera_histogramas_rgb(imagem_rgb[:, :, 0])
            hist_phi_vivos_g, hist_phi_mortos_g, hist_psi_vivos_g, hist_psi_mortos_g = gera_histogramas_rgb(imagem_rgb[:, :, 1])
            hist_phi_vivos_r, hist_phi_mortos_r, hist_psi_vivos_r, hist_psi_mortos_r = gera_histogramas_rgb(imagem_rgb[:, :, 2])

            #Concatena os histogramas dos 3 canais para cada uma das 4 situações após o jogo da vida (vivos->vivos, vivos->mortos, etc.)
            hist_phi_vivos = np.concatenate([np.concatenate([hist_phi_vivos_b,hist_phi_vivos_g]),hist_phi_vivos_r])
            hist_phi_mortos = np.concatenate([np.concatenate([hist_phi_mortos_b,hist_phi_mortos_g]),hist_phi_mortos_r])
            hist_psi_vivos = np.concatenate([np.concatenate([hist_psi_vivos_b,hist_psi_vivos_g]),hist_psi_vivos_r])
            hist_psi_mortos = np.concatenate([np.concatenate([hist_psi_mortos_b,hist_psi_mortos_g]),hist_psi_mortos_r])

            #salvando os histogramas
            index = os.path.splitext(os.path.basename(imagem_path))[0]
            file_path = os.path.join(f"{histograms_dir}/{classe}", f"{index}_phi_vivos_bgr.pkl")
            joblib.dump(hist_phi_vivos, file_path)
            file_path = os.path.join(f"{histograms_dir}/{classe}", f"{index}_phi_mortos_bgr.pkl")
            joblib.dump(hist_phi_mortos, file_path)
            file_path = os.path.join(f"{histograms_dir}/{classe}", f"{index}_psi_vivos_bgr.pkl")
            joblib.dump(hist_psi_vivos, file_path)
            file_path = os.path.join(f"{histograms_dir}/{classe}", f"{index}_psi_mortos_bgr.pkl")
            joblib.dump(hist_psi_mortos, file_path)
            progress_bar.update(1)

                
        progress_bar.close()


    print("Progresso concluído")

## Preparação de dados

In [15]:
def carrega_hist_rgb(imagem_path):
    #O objetivo dessa função é carregar os histogramas de determinada imagem

    #./data/dataset/images/classe/imagem.jpg
    imagem_path = imagem_path.split("/")
    dataset, classe, imagem = imagem_path[2], imagem_path[4], imagem_path[5]
    imagem = imagem.split(".")[0]

    histogramas_dir = f"./data/{dataset}/histograms_rgb"
    ##print(imagem_path)
    ##print(histogramas_dir)
    
    try:
        hist_phi_vivos = joblib.load(f"{histogramas_dir}/{classe}/{imagem}_phi_vivos_bgr.pkl")
        hist_phi_mortos = joblib.load(f"{histogramas_dir}/{classe}/{imagem}_phi_mortos_bgr.pkl")
        hist_psi_vivos = joblib.load(f"{histogramas_dir}/{classe}/{imagem}_psi_vivos_bgr.pkl")
        hist_psi_mortos = joblib.load(f"{histogramas_dir}/{classe}/{imagem}_psi_mortos_bgr.pkl")

        return hist_phi_vivos, hist_phi_mortos, hist_psi_vivos, hist_psi_mortos

    except:
        print("Os histogramas não foram encontrados. Verifique os diretórios e se os histogramas foram criados corretamente.")

In [16]:
def retorna_combinacao_rgb(imagem_path, combinacao):
    # O objetivo dessa funcao é criar combinacoes com os histogramas da imagem

    
    hist_phi_vivos, hist_phi_mortos, hist_psi_vivos, hist_psi_mortos = carrega_hist_rgb(imagem_path)
    
    switch = {
        0: hist_phi_vivos,
        1: hist_phi_mortos,
        2: hist_psi_vivos,
        3: hist_psi_mortos,
        4: np.concatenate([hist_phi_vivos, hist_phi_mortos]),   #histogramas phi
        5: np.concatenate([hist_psi_vivos, hist_psi_mortos]),   #histogramas psi
        6: np.concatenate([hist_phi_vivos, hist_psi_vivos]),    #histogramas vivos
        7: np.concatenate([hist_phi_mortos, hist_psi_mortos]),  #histogramas mortos
        8: np.concatenate([np.concatenate([hist_phi_vivos, hist_phi_mortos]),
                           np.concatenate([hist_psi_vivos, hist_psi_mortos])])
    }
    
    return switch.get(combinacao, None)

In [17]:
# Prepara os dados para treinar o modelo
def retorna_dados_rgb(images_dir): 
    histogramas_rgb = []
    rotulos_rgb = []
    print("Carregando histogramas")
    for classe in os.listdir(images_dir):
        dir_classe = f"{images_dir}/{classe}"
        classe = f"{classe}"
        
        lista_arquivos = os.listdir(dir_classe)
        num_arquivos = len(lista_arquivos)
        progress_bar = tqdm(total=num_arquivos, desc= classe)
        
        for imagem in os.listdir(dir_classe):
            imagem_path = f"{images_dir}/{classe}/{imagem}"
    
            #Pode alterar a combinacao 
            combinacao = retorna_combinacao_rgb(imagem_path, 8)
    
            #Lista dos histogramas
            histogramas_rgb.append(combinacao)
            #lista da classe assossiada ao histograma
            rotulos_rgb.append(classe)
            progress_bar.update(1)

        progress_bar.close()
    
    print("Progresso concluído")
    return histogramas_rgb, rotulos_rgb

# Funções p/ modelos

In [18]:
def gaussian(X_train, y_train, X_test, y_test):
    # Gaussian Naive Bayes
    gnb = GaussianNB()
    model1 = gnb.fit(X_train, y_train)
    aux = gnb.predict(X_test)
    print(classification_report(y_test, aux, zero_division=1))
    #print(confusion_matrix(y_test, aux))

def logistic_regr(X_train, y_train, X_test, y_test):
    # Logistic Regression
    logreg = LogisticRegression(max_iter = 1000)
    model2 = logreg.fit(X_train, y_train)
    aux = logreg.predict(X_test)
    print(classification_report(y_test, aux, zero_division=1))
    #print(confusion_matrix(y_test, aux))

def decision_tree(X_train, y_train, X_test, y_test):
    # Decision Tree
    dectree = DecisionTreeClassifier()
    model3 = dectree.fit(X_train, y_train)
    aux = dectree.predict(X_test)
    print(classification_report(y_test, aux, zero_division=1))
    #print(confusion_matrix(y_test, aux))

def knn(X_train, y_train, X_test, y_test):
    # K-Nearest Neighbors
    knn = KNeighborsClassifier(n_neighbors = 3)
    model4 = knn.fit(X_train, y_train)
    aux = knn.predict(X_test)
    print(classification_report(y_test, aux, zero_division=1))
    #print(confusion_matrix(y_test, aux))

def lda(X_train, y_train, X_test, y_test):
    # Linear Discriminant Analysis
    lda = LinearDiscriminantAnalysis()
    model5 = lda.fit(X_train, y_train)
    aux = lda.predict(X_test)
    print(classification_report(y_test, aux, zero_division=1))
    #print(confusion_matrix(y_test, aux))

def svm(X_train, y_train, X_test, y_test):
    # Support Vector Machine
    svm = SVC()
    model6 = svm.fit(X_train, y_train)
    aux = svm.predict(X_test)
    print(classification_report(y_test, aux, zero_division=1))
    #print(confusion_matrix(y_test, aux))

def random_forest(X_train, y_train, X_test, y_test):
    # RandomForest
    rf = RandomForestClassifier()
    model7 = rf.fit(X_train, y_train)
    aux = rf.predict(X_test)
    print(classification_report(y_test, aux, zero_division=1))
    #print(confusion_matrix(y_test, aux))

def neural_net(X_train, y_train, X_test, y_test):
    # Neural Net
    nnet = MLPClassifier(alpha=1)
    model8 = nnet.fit(X_train, y_train)
    aux = nnet.predict(X_test)
    print(classification_report(y_test, aux, zero_division=1))
    #print(confusion_matrix(y_test, aux))

# Dataset Corel 1k

## Em escala de cinza:

### Gerando histogramas

In [ ]:
#gerador_histogramas(dataset_dir, 128)

### Testando modelos de aprendizado de máquina combinando histogramas

In [ ]:
histogramas, rotulos = retorna_dados(images_dir)
X = np.array(histogramas)
y = np.array(rotulos)

#Dividindo treino e teste na respectiva proporção (0.8 / 0.2), stratify = y mantém uma proporção entre os dados de teste de cada classe
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1,stratify=y)

In [ ]:
#Utilizando tecnicas de normalizacao
# 1 = MinMaxScaler, 2 = StandardScaler, 3 = MaxAbsScaler, 4 = RobustScaler
selectedNormalization = 1

if selectedNormalization == 1:
    scaler = preprocessing.MinMaxScaler()
if selectedNormalization == 2:
    scaler = preprocessing.StandardScaler()
if selectedNormalization == 3:
    scaler = preprocessing.MaxAbsScaler()
if selectedNormalization == 4:
    scaler = preprocessing.RobustScaler()

# Escalando os dados de treinamento
X_train = scaler.fit_transform(X_train)
# Escalando os dados de teste com os dados de treinamento, visto que os dados de teste podem ser apenas 1 amostra
X_test = scaler.transform(X_test)

In [ ]:
gaussian(X_train, y_train, X_test, y_test)

In [ ]:
logistic_regr(X_train, y_train, X_test, y_test)

In [ ]:
decision_tree(X_train, y_train, X_test, y_test)

In [ ]:
knn(X_train, y_train, X_test, y_test)

In [ ]:
lda(X_train, y_train, X_test, y_test)

In [ ]:
svm(X_train, y_train, X_test, y_test)

In [ ]:
random_forest(X_train, y_train, X_test, y_test)

In [ ]:
neural_net(X_train, y_train, X_test, y_test)

## Em RGB:

### Gerando histogramas

In [ ]:
gerador_histogramas_RGB(dataset_dir, 128)

### Testando modelos de aprendizado

In [ ]:
histogramas_rgb, rotulos_rgb = retorna_dados_rgb(images_dir)
X = np.array(histogramas_rgb)
y = np.array(rotulos_rgb)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)

In [ ]:
#Utilizando tecnicas de normalizacao
# 1 = MinMaxScaler, 2 = StandardScaler, 3 = MaxAbsScaler, 4 = RobustScaler
selectedNormalization = 1

if selectedNormalization == 1:
    scaler = preprocessing.MinMaxScaler()
if selectedNormalization == 2:
    scaler = preprocessing.StandardScaler()
if selectedNormalization == 3:
    scaler = preprocessing.MaxAbsScaler()
if selectedNormalization == 4:
    scaler = preprocessing.RobustScaler()

# Escalando os dados de treinamento
X_train = scaler.fit_transform(X_train)
# Escalando os dados de teste com os dados de treinamento, visto que os dados de teste podem ser apenas 1 amostra
X_test = scaler.transform(X_test)

In [ ]:
gaussian(X_train, y_train, X_test, y_test)

In [ ]:
logistic_regr(X_train, y_train, X_test, y_test)

In [ ]:
decision_tree(X_train, y_train, X_test, y_test)

In [ ]:
knn(X_train, y_train, X_test, y_test)

In [ ]:
lda(X_train, y_train, X_test, y_test)

In [ ]:
svm(X_train, y_train, X_test, y_test)

In [ ]:
random_forest(X_train, y_train, X_test, y_test)

In [ ]:
neural_net(X_train, y_train, X_test, y_test)

# Dataset_1 Caltech256

## Em RBG:


### Gerando histogramas

In [ ]:
#gerador_histogramas_RGB(dataset_1_dir, 128)

### Testando modelos de aprendizado 

In [ ]:
histogramas_rgb, rotulos_rgb = retorna_dados_rgb(images_1_dir)
X = np.array(histogramas_rgb)
y = np.array(rotulos_rgb)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)

In [ ]:
#Utilizando tecnicas de normalizacao
# 1 = MinMaxScaler, 2 = StandardScaler, 3 = MaxAbsScaler, 4 = RobustScaler
selectedNormalization = 1

if selectedNormalization == 1:
    scaler = preprocessing.MinMaxScaler()
if selectedNormalization == 2:
    scaler = preprocessing.StandardScaler()
if selectedNormalization == 3:
    scaler = preprocessing.MaxAbsScaler()
if selectedNormalization == 4:
    scaler = preprocessing.RobustScaler()

# Escalando os dados de treinamento
X_train = scaler.fit_transform(X_train)
# Escalando os dados de teste com os dados de treinamento, visto que os dados de teste podem ser apenas 1 amostra
X_test = scaler.transform(X_test)

In [ ]:
gaussian(X_train, y_train, X_test, y_test)

In [ ]:
logistic_regr(X_train, y_train, X_test, y_test)

In [ ]:
decision_tree(X_train, y_train, X_test, y_test)

In [ ]:
knn(X_train, y_train, X_test, y_test)

In [ ]:
lda(X_train, y_train, X_test, y_test)

In [ ]:
svm(X_train, y_train, X_test, y_test)

In [ ]:
random_forest(X_train, y_train, X_test, y_test)

In [ ]:
neural_net(X_train, y_train, X_test, y_test)

## Em escala de cinza:

### Gerando histogramas

In [ ]:
#gerador_histogramas(dataset_1_dir, 128)

### Testando modelos de aprendizado

In [ ]:
histogramas, rotulos = retorna_dados(images_1_dir)
X = np.array(histogramas)
y = np.array(rotulos)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1,stratify=y)

In [ ]:
#Utilizando tecnicas de normalizacao
# 1 = MinMaxScaler, 2 = StandardScaler, 3 = MaxAbsScaler, 4 = RobustScaler
selectedNormalization = 1

if selectedNormalization == 1:
    scaler = preprocessing.MinMaxScaler()
if selectedNormalization == 2:
    scaler = preprocessing.StandardScaler()
if selectedNormalization == 3:
    scaler = preprocessing.MaxAbsScaler()
if selectedNormalization == 4:
    scaler = preprocessing.RobustScaler()

# Escalando os dados de treinamento
X_train = scaler.fit_transform(X_train)
# Escalando os dados de teste com os dados de treinamento, visto que os dados de teste podem ser apenas 1 amostra
X_test = scaler.transform(X_test)

In [ ]:
gaussian(X_train, y_train, X_test, y_test)

In [ ]:
logistic_regr(X_train, y_train, X_test, y_test)

In [ ]:
decision_tree(X_train, y_train, X_test, y_test)

In [ ]:
knn(X_train, y_train, X_test, y_test)

In [ ]:
lda(X_train, y_train, X_test, y_test)

In [ ]:
svm(X_train, y_train, X_test, y_test)

In [ ]:
random_forest(X_train, y_train, X_test, y_test)

In [ ]:
neural_net(X_train, y_train, X_test, y_test)

# Dataset_2 CIFAR 100

## Em escala de cinza:

### Gerando histogramas

In [ ]:
#gerador_histogramas(dataset_2_dir, 32)

### Testando modelos de aprendizado

In [ ]:
histogramas, rotulos = retorna_dados(images_2_dir)
X = np.array(histogramas)
y = np.array(rotulos)

#Dividindo treino e teste na respectiva proporção (0.8 / 0.2), stratify = y mantém uma proporção entre os dados de teste de cada classe
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1,stratify=y)

In [ ]:
#Utilizando tecnicas de normalizacao
# 1 = MinMaxScaler, 2 = StandardScaler, 3 = MaxAbsScaler, 4 = RobustScaler
selectedNormalization = 1

if selectedNormalization == 1:
    scaler = preprocessing.MinMaxScaler()
if selectedNormalization == 2:
    scaler = preprocessing.StandardScaler()
if selectedNormalization == 3:
    scaler = preprocessing.MaxAbsScaler()
if selectedNormalization == 4:
    scaler = preprocessing.RobustScaler()

# Escalando os dados de treinamento
X_train = scaler.fit_transform(X_train)
# Escalando os dados de teste com os dados de treinamento, visto que os dados de teste podem ser apenas 1 amostra
X_test = scaler.transform(X_test)

In [ ]:
gaussian(X_train, y_train, X_test, y_test)

In [ ]:
logistic_regr(X_train, y_train, X_test, y_test)

In [ ]:
decision_tree(X_train, y_train, X_test, y_test)

In [ ]:
knn(X_train, y_train, X_test, y_test)

In [ ]:
lda(X_train, y_train, X_test, y_test)

In [ ]:
svm(X_train, y_train, X_test, y_test)

In [ ]:
random_forest(X_train, y_train, X_test, y_test)

In [ ]:
neural_net(X_train, y_train, X_test, y_test)

## Em RGB:

### Gerando histogramas

In [ ]:
gerador_histogramas_RGB(dataset_2_dir, 32) #escala em 32 pq o dataset vem em 32x32

### Testando modelos

In [ ]:
histogramas_rgb, rotulos_rgb = retorna_dados_rgb(images_2_dir)
X = np.array(histogramas_rgb)
y = np.array(rotulos_rgb)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)

In [ ]:
#Utilizando tecnicas de normalizacao
# 1 = MinMaxScaler, 2 = StandardScaler, 3 = MaxAbsScaler, 4 = RobustScaler
selectedNormalization = 1

if selectedNormalization == 1:
    scaler = preprocessing.MinMaxScaler()
if selectedNormalization == 2:
    scaler = preprocessing.StandardScaler()
if selectedNormalization == 3:
    scaler = preprocessing.MaxAbsScaler()
if selectedNormalization == 4:
    scaler = preprocessing.RobustScaler()

# Escalando os dados de treinamento
X_train = scaler.fit_transform(X_train)
# Escalando os dados de teste com os dados de treinamento, visto que os dados de teste podem ser apenas 1 amostra
X_test = scaler.transform(X_test)

In [ ]:
gaussian(X_train, y_train, X_test, y_test)

In [ ]:
logistic_regr(X_train, y_train, X_test, y_test)

In [ ]:
decision_tree(X_train, y_train, X_test, y_test)

In [ ]:
knn(X_train, y_train, X_test, y_test)

In [ ]:
lda(X_train, y_train, X_test, y_test)

In [ ]:
svm(X_train, y_train, X_test, y_test)

In [ ]:
random_forest(X_train, y_train, X_test, y_test)

In [ ]:
neural_net(X_train, y_train, X_test, y_test)

# Dataset_3 CIFAR-10

## Em escala de cinza:

### Gerando histogramas

In [ ]:
gerador_histogramas(dataset_3_dir, 32)

### Testando os modelos

In [ ]:
histogramas, rotulos = retorna_dados(images_3_dir)
X = np.array(histogramas)
y = np.array(rotulos)

#Dividindo treino e teste na respectiva proporção (0.8 / 0.2), stratify = y mantém uma proporção entre os dados de teste de cada classe
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1,stratify=y)

In [ ]:
#Utilizando tecnicas de normalizacao
# 1 = MinMaxScaler, 2 = StandardScaler, 3 = MaxAbsScaler, 4 = RobustScaler
selectedNormalization = 1

if selectedNormalization == 1:
    scaler = preprocessing.MinMaxScaler()
if selectedNormalization == 2:
    scaler = preprocessing.StandardScaler()
if selectedNormalization == 3:
    scaler = preprocessing.MaxAbsScaler()
if selectedNormalization == 4:
    scaler = preprocessing.RobustScaler()

# Escalando os dados de treinamento
X_train = scaler.fit_transform(X_train)
# Escalando os dados de teste com os dados de treinamento, visto que os dados de teste podem ser apenas 1 amostra
X_test = scaler.transform(X_test)

In [ ]:
gaussian(X_train, y_train, X_test, y_test)

In [ ]:
logistic_regr(X_train, y_train, X_test, y_test)

In [ ]:
decision_tree(X_train, y_train, X_test, y_test)

In [ ]:
knn(X_train, y_train, X_test, y_test)

In [ ]:
lda(X_train, y_train, X_test, y_test)

In [ ]:
svm(X_train, y_train, X_test, y_test)

In [ ]:
random_forest(X_train, y_train, X_test, y_test)

In [ ]:
neural_net(X_train, y_train, X_test, y_test)

## Em RGB:

### Gerando histogramas

In [19]:
gerador_histogramas_RGB(dataset_3_dir, 32) #escala em 32 pq o dataset vem em 32x32

Gerando histogramas


truck: 100%|████████████████████████████████████████████████████████████████| 6000/6000 [16:11<00:00,  6.18it/s]


Progresso concluído


### Testando modelos

In [21]:
histogramas_rgb, rotulos_rgb = retorna_dados_rgb(images_3_dir)
X = np.array(histogramas_rgb)
y = np.array(rotulos_rgb)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)

Carregando histogramas


truck: 100%|███████████████████████████████████████████████████████████████| 6000/6000 [00:12<00:00, 491.71it/s]


Progresso concluído


In [22]:
#Utilizando tecnicas de normalizacao
# 1 = MinMaxScaler, 2 = StandardScaler, 3 = MaxAbsScaler, 4 = RobustScaler
selectedNormalization = 1

if selectedNormalization == 1:
    scaler = preprocessing.MinMaxScaler()
if selectedNormalization == 2:
    scaler = preprocessing.StandardScaler()
if selectedNormalization == 3:
    scaler = preprocessing.MaxAbsScaler()
if selectedNormalization == 4:
    scaler = preprocessing.RobustScaler()

# Escalando os dados de treinamento
X_train = scaler.fit_transform(X_train)
# Escalando os dados de teste com os dados de treinamento, visto que os dados de teste podem ser apenas 1 amostra
X_test = scaler.transform(X_test)

In [23]:
gaussian(X_train, y_train, X_test, y_test)

              precision    recall  f1-score   support

    airplane       0.42      0.32      0.36      1200
  automobile       0.23      0.05      0.09      1200
        bird       0.30      0.12      0.17      1200
         cat       0.23      0.08      0.12      1200
        deer       0.23      0.35      0.28      1200
         dog       0.22      0.12      0.15      1200
        frog       0.20      0.55      0.30      1200
       horse       0.24      0.04      0.07      1200
        ship       0.28      0.24      0.26      1200
       truck       0.22      0.52      0.31      1200

    accuracy                           0.24     12000
   macro avg       0.26      0.24      0.21     12000
weighted avg       0.26      0.24      0.21     12000



In [24]:
logistic_regr(X_train, y_train, X_test, y_test)

              precision    recall  f1-score   support

    airplane       0.40      0.40      0.40      1200
  automobile       0.28      0.30      0.29      1200
        bird       0.29      0.21      0.24      1200
         cat       0.23      0.17      0.19      1200
        deer       0.28      0.35      0.31      1200
         dog       0.25      0.24      0.25      1200
        frog       0.34      0.47      0.40      1200
       horse       0.28      0.18      0.22      1200
        ship       0.36      0.40      0.38      1200
       truck       0.31      0.37      0.34      1200

    accuracy                           0.31     12000
   macro avg       0.30      0.31      0.30     12000
weighted avg       0.30      0.31      0.30     12000



In [25]:
decision_tree(X_train, y_train, X_test, y_test)

              precision    recall  f1-score   support

    airplane       0.29      0.27      0.28      1200
  automobile       0.20      0.21      0.21      1200
        bird       0.17      0.17      0.17      1200
         cat       0.14      0.14      0.14      1200
        deer       0.21      0.21      0.21      1200
         dog       0.14      0.14      0.14      1200
        frog       0.26      0.27      0.27      1200
       horse       0.15      0.15      0.15      1200
        ship       0.24      0.23      0.23      1200
       truck       0.22      0.23      0.22      1200

    accuracy                           0.20     12000
   macro avg       0.20      0.20      0.20     12000
weighted avg       0.20      0.20      0.20     12000



In [26]:
knn(X_train, y_train, X_test, y_test)

              precision    recall  f1-score   support

    airplane       0.34      0.40      0.37      1200
  automobile       0.22      0.41      0.29      1200
        bird       0.17      0.25      0.21      1200
         cat       0.15      0.22      0.18      1200
        deer       0.24      0.23      0.24      1200
         dog       0.17      0.10      0.12      1200
        frog       0.28      0.39      0.32      1200
       horse       0.26      0.08      0.12      1200
        ship       0.46      0.16      0.24      1200
       truck       0.30      0.12      0.17      1200

    accuracy                           0.24     12000
   macro avg       0.26      0.24      0.23     12000
weighted avg       0.26      0.24      0.23     12000



In [27]:
lda(X_train, y_train, X_test, y_test)

              precision    recall  f1-score   support

    airplane       0.40      0.35      0.37      1200
  automobile       0.26      0.29      0.27      1200
        bird       0.26      0.18      0.21      1200
         cat       0.19      0.17      0.18      1200
        deer       0.28      0.35      0.31      1200
         dog       0.22      0.23      0.23      1200
        frog       0.32      0.39      0.35      1200
       horse       0.19      0.15      0.17      1200
        ship       0.32      0.32      0.32      1200
       truck       0.28      0.31      0.30      1200

    accuracy                           0.28     12000
   macro avg       0.27      0.28      0.27     12000
weighted avg       0.27      0.28      0.27     12000



In [ ]:
svm(X_train, y_train, X_test, y_test)

In [ ]:
random_forest(X_train, y_train, X_test, y_test)

In [ ]:
neural_net(X_train, y_train, X_test, y_test)